# [SOLUTION] Exercise - Output structured Agent responses

In this exercise, you'll learn how to enhance your AI agent to provide structured outputs using Pydantic models. This will help ensure the agent's responses are consistent, validated, and easily usable in downstream applications.

## Challenge

You have an existing Agent class that can:
- Process user messages
- Use tools when needed
- Generate responses

Now you need to enhance it to:
- Define structured output formats using Pydantic
- Parse and validate responses
- Return data in a consistent JSON format



## Setup
First, let's import the necessary libraries:

In [2]:
from typing import List, Any, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM
from lib.parsers import PydanticOutputParser, JsonOutputParser

## Defining Structured Output Models

Let's create a Pydantic model for a meeting summary with action items:


In [3]:
class ActionItem(BaseModel):
    """Represents a single action item from a meeting"""
    task: Annotated[str, Field(description="The task to be completed")]
    assignee: Annotated[str, Field(description="Person responsible for the task")]
    due_date: Annotated[str, Field(description="When the task should be completed")]

In [4]:
class MeetingSummary(BaseModel):
    """A structured summary of a meeting"""
    title: Annotated[str, Field(description="Title of the meeting")]
    date: Annotated[str, Field(description="Date when the meeting occurred")]
    participants: Annotated[List[str], Field(description="List of meeting attendees")]
    key_points: Annotated[List[str], Field(description="Main points discussed in the meeting")]
    action_items: Annotated[List[ActionItem], Field(description="List of action items from the meeting")]

## Enhanced Agent Class

Now let's create an enhanced version of our Agent class that supports structured outputs:


In [5]:
class StructuredAgent:
    """An AI Agent that provides structured outputs"""
    
    def __init__(
        self,
        role: str = "Meeting Assistant",
        instructions: str = "Help summarize meetings and track action items",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None,
        output_model: BaseModel = None
    ):
        """Initialize the agent with its configuration
        
        Args:
            role: The agent's role/persona
            instructions: Basic instructions for the agent
            model: The LLM model to use
            temperature: Creativity parameter (0.0 = more deterministic)
            tools: List of tools the agent can use
            output_model: Pydantic model for structured output
        """
        self.model = model
        self.role = role
        self.instructions = instructions
        self.tools = tools
        self.output_model = output_model

        # Load environment variables
        load_dotenv()
        
        # Initialize the LLM
        self.llm = LLM(
            model=model,
            temperature=temperature,
            tools=tools,
        )

    def invoke(self, user_message: str) -> dict:
        """Process a user message and return a structured response
        
        Args:
            user_message: The user's input message
            
        Returns:
            A dictionary containing the structured response
        """
        messages = [
            SystemMessage(
                content=(
                    f"You're an AI Agent and your role is {self.role}. "  
                    f"Your instructions: {self.instructions}"
                )
            )
        ]
        # Add user message
        messages.append(UserMessage(content=user_message))
        
        # Get AI response with structured format
        if self.output_model:
            ai_message = self.llm.invoke(input=messages, response_format=self.output_model)
            parser = JsonOutputParser()
            return parser.parse(ai_message)
        else:
            # Handle unstructured response if no model specified
            ai_message = self.llm.invoke(messages)
            return {"response": ai_message.content}


## Testing the Structured Agent

Let's test our enhanced agent with a meeting summary example:


In [6]:
# Create an agent instance with the MeetingSummary model
meeting_agent = StructuredAgent(
    role="Meeting Assistant",
    instructions="Summarize meetings and track action items in a structured format",
    output_model=MeetingSummary
)

In [7]:
meeting_transcript = """
Project Planning Meeting - March 15, 2024

Attendees: John, Sarah, Mike

Discussion:
- Reviewed Q1 project timeline
- Discussed resource allocation
- Identified potential risks

Next steps:
1. John will update the project plan by next Friday
2. Sarah needs to coordinate with the design team by Wednesday
3. Mike will prepare the risk assessment document by end of month
"""

In [8]:
summary = meeting_agent.invoke(meeting_transcript)
print(json.dumps(summary, indent=2))

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: voc-1454**************************************6929. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## Validating the Output

Let's verify that our output matches our Pydantic model structure:


In [12]:
# Create a MeetingSummary instance from the output
validated_summary = MeetingSummary(**summary)

In [13]:
# Access structured data
print("Meeting Title:", validated_summary.title)
print("\nParticipants:")
for participant in validated_summary.participants:
    print(f"- {participant}")

print("\nAction Items:")
for item in validated_summary.action_items:
    print(f"- {item.task} (Assigned to: {item.assignee}, Due: {item.due_date})")

Meeting Title: Project Planning Meeting

Participants:
- John
- Sarah
- Mike

Action Items:
- Update the project plan (Assigned to: John, Due: 2024-03-22)
- Coordinate with the design team (Assigned to: Sarah, Due: 2024-03-20)
- Prepare the risk assessment document (Assigned to: Mike, Due: 2024-03-31)
